In [1]:
import os
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

sys.path.append('../')

from tsgen import TsGenerator

tsgen = TsGenerator()

Declaring class instance from: S3-FRBR:Nc_Explorer
Input NetCDF file folde not set. Proceed at your own risk.


## Defining the working directories

* Folder of files containing raw pixels extracted with GPT
* Folder to save the processed files
* Folder in which the plots should be saved

In [2]:
# in_dir = 'D:\\processing\\win\\MANACAPURU'
in_dir = 'D:\\S3\\L2_WFR_subset\\MADEIRA'
out_dir = 'D:\\processing\\win\\MADEIRAv9b'
img_dir = 'D:\\processing\\win\\a1_plots\\MADEIRAv9b'

In [3]:
todo = tsgen.build_list_from_subset(in_dir)
todo_fullpath = [os.path.join(in_dir,csv) for csv in todo]
print(f'{todo[0]}\n\n{todo_fullpath[0]}')

S3A_OL_2_WFR____20160425T134227_20160425T134427_20171030T154612_0119_003_238______MR1_R_NT_002_subset.txt

D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20160425T134227_20160425T134427_20171030T154612_0119_003_238______MR1_R_NT_002_subset.txt


## Generating new CSVs and plots based in the RAW data.

In [4]:
t1 = time.perf_counter()
skiplst = []
donelst = []

total = len(todo_fullpath)
for n,img in enumerate(todo_fullpath):
    print(f'>>> Processing: {n+1} of {total} ... {img}')
    
    # reprocessing the raw CSVs and removing 
    # reflectances above the threshold in IR.
    
    
    try:
        dfpth, df = tsgen.update_csvs(csv_path=img,savepath=out_dir,threshold=0.2)
    except Exception as e:
        print("type error: " + str(e))
        skiplst.append(img)
        continue

    # The KDE needs at least two different reflectance values to work.
    if dfpth == 'KDE_fail':
        skiplst.append(img)
        continue
    
#     try:
#         dfpth, df = tsgen.update_csvs(csv_path=img,savepath=out_dir,threshold=0.2)
#     except Exception as e:
#         print("type error: " + str(e))
#         print(f'Skipping CSV for lack of data: {figdate}')
#         skiplst.append(img)
#         continue

#     dfpth, df = tsgen.update_csvs(csv_path=img,
#                                   savepath=out_dir,
#                                   threshold=0.2)
    
    if len(df) < 1:
        print(f'Skipping empty CSV: {dfpth}')
        skiplst.append(img)
        continue
    else:
        donelst.append(img)
    
    
    # fig params -------------------------------------------------
    figdate = os.path.basename(img).split('____')[1].split('_')[0]
    figtitl = os.path.basename(out_dir)+'_'+figdate
    savpt = os.path.join(img_dir,figdate+'.png')
    
    print(f'Generating image: {savpt}')
    
    # generate plot v1 --------------------------
#     tsgen.plot_single_sktr(
#         xdata=df['Oa08_reflectance:float'],
#         ydata=df['Oa21_reflectance:float'],
#         xlabel='Oa08: 665 nm',
#         ylabel='Oa17: 865 nm',
#         color=df['T865:float'],
#         clabel='T865',
#         title=figtitl,
#         savepathname=savpt)
    
    # generate plot v3 --------------------------
    tsgen.plot_sidebyside_sktr(x1_data=df['Oa08_reflectance:float'],
                               y1_data=df['Oa17_reflectance:float'],
                               x2_data=df['Oa08_reflectance:float'],
                               y2_data=df['Oa17_reflectance:float'],
                               x_lbl='RED: Oa08 (665nm)',
                               y_lbl='NIR: Oa17 (865nm)',
                               c1_data=df['A865:float'],
                               c1_lbl='Aer. Angstrom Expoent (A865)',
                               c2_data=df['T865:float'],
                               c2_lbl='Aer. Optical Thickness (T865)',
                               title=f'{os.path.basename(out_dir)} WFR {figdate} RED:Oa08(665nm) x NIR:Oa17(865nm)',
                               savepathname=savpt)
    
    
t2 = time.perf_counter()
print(f'>>> Finished in {round(t2 - t1, 2)} second(s). <<<')

>>> Processing: 1 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20160425T134227_20160425T134427_20171030T154612_0119_003_238______MR1_R_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20160425T134227_20160425T134427_20171030T154612_0119_003_238______MR1_R_NT_002_subset.txt
Skipping empty CSV: D:\processing\win\MADEIRAv9b\S3A_OL_2_WFR____20160425T134227_20160425T134427_20171030T154612_0119_003_238______MR1_R_NT_002_subset.txt
>>> Processing: 2 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20160428T140436_20160428T140636_20171030T201052_0119_003_281______MR1_R_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20160428T140436_20160428T140636_20171030T201052_0119_003_281______MR1_R_NT_002_subset.txt
Skipping empty CSV: D:\processing\win\MADEIRAv9b\S3A_OL_2_WFR____20160428T140436_20160428T140636_20171030T201052_0119_003_281______MR1_R_NT_002_subset.txt
>>> Processing: 3 of 1761 ... D:\S3\L2_WFR_sub

Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20160603T133124_20160603T133324_20171101T032646_0119_005_024______MR1_R_NT_002_subset.txt
Generating image: D:\processing\win\a1_plots\MADEIRAv9b\20160603T133124.png
>>> Processing: 20 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20160606T135333_20160606T135533_20171101T050305_0119_005_067______MR1_R_NT_002_subset.txt
Calculating GLINT column...
Saving dataset: D:\processing\win\MADEIRAv9b\S3A_OL_2_WFR____20160606T135333_20160606T135533_20171101T050305_0119_005_067______MR1_R_NT_002_subset.txt
Generating image: D:\processing\win\a1_plots\MADEIRAv9b\20160606T135333.png
>>> Processing: 21 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20160607T132743_20160607T132943_20171101T053245_0119_005_081______MR1_R_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20160607T132743_20160607T132943_20171101T053245_0119_005_081______MR1_R_NT_002_subset.txt
Skipping empty CSV: D:\

Calculating GLINT column...
Saving dataset: D:\processing\win\MADEIRAv9b\S3A_OL_2_WFR____20160719T133847_20160719T134047_20171102T035456_0119_006_295______MR1_R_NT_002_subset.txt
Generating image: D:\processing\win\a1_plots\MADEIRAv9b\20160719T133847.png
>>> Processing: 40 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20160722T140056_20160722T140256_20171102T053354_0119_006_338______MR1_R_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20160722T140056_20160722T140256_20171102T053354_0119_006_338______MR1_R_NT_002_subset.txt
Skipping empty CSV: D:\processing\win\MADEIRAv9b\S3A_OL_2_WFR____20160722T140056_20160722T140256_20171102T053354_0119_006_338______MR1_R_NT_002_subset.txt
>>> Processing: 41 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20160723T133506_20160723T133706_20171102T060548_0119_006_352______MR1_R_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20160723T133506_20160723T133706_20

>>> Processing: 59 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20160831T132251_20160831T132451_20171103T032908_0119_008_138______MR1_R_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20160831T132251_20160831T132451_20171103T032908_0119_008_138______MR1_R_NT_002_subset.txt
Skipping empty CSV: D:\processing\win\MADEIRAv9b\S3A_OL_2_WFR____20160831T132251_20160831T132451_20171103T032908_0119_008_138______MR1_R_NT_002_subset.txt
>>> Processing: 60 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20160903T134540_20160903T134740_20171103T050919_0119_008_181______MR1_R_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20160903T134540_20160903T134740_20171103T050919_0119_008_181______MR1_R_NT_002_subset.txt
Skipping empty CSV: D:\processing\win\MADEIRAv9b\S3A_OL_2_WFR____20160903T134540_20160903T134740_20171103T050919_0119_008_181______MR1_R_NT_002_subset.txt
>>> Processing: 61 of 1761 ... D:\S3\L2_WFR_

Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20161012T133521_20161012T133721_20171104T023856_0119_009_352______MR1_R_NT_002_subset.txt
Skipping empty CSV: D:\processing\win\MADEIRAv9b\S3A_OL_2_WFR____20161012T133521_20161012T133721_20171104T023856_0119_009_352______MR1_R_NT_002_subset.txt
>>> Processing: 79 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20161015T135609_20161015T135809_20171104T041807_0119_010_010______MR1_R_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20161015T135609_20161015T135809_20171104T041807_0119_010_010______MR1_R_NT_002_subset.txt
Skipping empty CSV: D:\processing\win\MADEIRAv9b\S3A_OL_2_WFR____20161015T135609_20161015T135809_20171104T041807_0119_010_010______MR1_R_NT_002_subset.txt
>>> Processing: 80 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20161016T133004_20161016T133204_20171104T044752_0119_010_024______MR1_R_NT_002_subset.txt
Calculating GLINT column...
Skipping empty d

...

Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20191224T135343_20191224T135643_20191230T151313_0180_053_067_3060_MAR_O_NT_002_subset.txt
Skipping empty CSV: D:\processing\win\MADEIRAv9b\S3A_OL_2_WFR____20191224T135343_20191224T135643_20191230T151313_0180_053_067_3060_MAR_O_NT_002_subset.txt
>>> Processing: 713 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20191225T132732_20191225T133032_20191231T095433_0179_053_081_3060_MAR_O_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20191225T132732_20191225T133032_20191231T095433_0179_053_081_3060_MAR_O_NT_002_subset.txt
Skipping empty CSV: D:\processing\win\MADEIRAv9b\S3A_OL_2_WFR____20191225T132732_20191225T133032_20191231T095433_0179_053_081_3060_MAR_O_NT_002_subset.txt
>>> Processing: 714 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3B_OL_2_WFR____20191226T140251_20191226T140551_20200103T064308_0179_033_338_3060_MAR_O_NT_002_subset.txt
Calculating GLINT column...
Skipping empty

Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20200113T133503_20200113T133803_20200114T222122_0179_053_352_3060_MAR_O_NT_002_subset.txt
Skipping empty CSV: D:\processing\win\MADEIRAv9b\S3A_OL_2_WFR____20200113T133503_20200113T133803_20200114T222122_0179_053_352_3060_MAR_O_NT_002_subset.txt
>>> Processing: 731 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3B_OL_2_WFR____20200115T134408_20200115T134708_20200118T222049_0180_034_238_3060_MAR_O_NT_002_subset.txt
Calculating GLINT column...
Saving dataset: D:\processing\win\MADEIRAv9b\S3B_OL_2_WFR____20200115T134408_20200115T134708_20200118T222049_0180_034_238_3060_MAR_O_NT_002_subset.txt
Generating image: D:\processing\win\a1_plots\MADEIRAv9b\20200115T134408.png
>>> Processing: 732 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20200116T135730_20200116T140030_20200117T205932_0179_054_010_3060_MAR_O_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20200116T135730_20200116T140030_

>>> Processing: 748 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3B_OL_2_WFR____20200204T132523_20200204T132823_20200205T184924_0179_035_138_3060_MAR_O_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3B_OL_2_WFR____20200204T132523_20200204T132823_20200205T184924_0179_035_138_3060_MAR_O_NT_002_subset.txt
Skipping empty CSV: D:\processing\win\MADEIRAv9b\S3B_OL_2_WFR____20200204T132523_20200204T132823_20200205T184924_0179_035_138_3060_MAR_O_NT_002_subset.txt
>>> Processing: 749 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20200204T140459_20200204T140759_20200205T224626_0179_054_281_3060_MAR_O_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20200204T140459_20200204T140759_20200205T224626_0179_054_281_3060_MAR_O_NT_002_subset.txt
Skipping empty CSV: D:\processing\win\MADEIRAv9b\S3A_OL_2_WFR____20200204T140459_20200204T140759_20200205T224626_0179_054_281_3060_MAR_O_NT_002_subset.txt
>>> Processing: 750 of 1761 ... D:\S3\L2_W

Calculating GLINT column...
Skipping empty dataset: S3B_OL_2_WFR____20200223T133255_20200223T133555_20200224T220719_0179_036_024_3060_MAR_O_NT_002_subset.txt
Skipping empty CSV: D:\processing\win\MADEIRAv9b\S3B_OL_2_WFR____20200223T133255_20200223T133555_20200224T220719_0179_036_024_3060_MAR_O_NT_002_subset.txt
>>> Processing: 766 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20200224T134616_20200224T134916_20200225T211517_0179_055_181_3060_MAR_O_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20200224T134616_20200224T134916_20200225T211517_0179_055_181_3060_MAR_O_NT_002_subset.txt
Generating image: D:\processing\win\a1_plots\MADEIRAv9b\20200224T134616.png
>>> Processing: 767 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3B_OL_2_WFR____20200226T135521_20200226T135821_20200302T071746_0179_036_067_3060_MAR_O_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3B_OL_2_WFR____20200226T135521_20200226T135821_20200302T071746_0179_

Calculating GLINT column...
Skipping empty dataset: S3B_OL_2_WFR____20200317T133641_20200317T133941_20200318T201158_0179_036_352_3060_MAR_O_NT_002_subset.txt
Generating image: D:\processing\win\a1_plots\MADEIRAv9b\20200317T133641.png
>>> Processing: 785 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20200318T135001_20200318T135301_20200319T210614_0179_056_124_3060_MAR_O_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20200318T135001_20200318T135301_20200319T210614_0179_056_124_3060_MAR_O_NT_002_subset.txt
Skipping empty CSV: D:\processing\win\MADEIRAv9b\S3A_OL_2_WFR____20200318T135001_20200318T135301_20200319T210614_0179_056_124_3060_MAR_O_NT_002_subset.txt
>>> Processing: 786 of 1761 ... D:\S3\L2_WFR_subset\MADEIRA\S3A_OL_2_WFR____20200319T132350_20200319T132650_20200320T220408_0180_056_138_3060_MAR_O_NT_002_subset.txt
Calculating GLINT column...
Skipping empty dataset: S3A_OL_2_WFR____20200319T132350_20200319T132650_20200320T220408_0180_

## Generating mean values dataset

In [5]:
wdir = out_dir

todo = tsgen.build_list_from_subset(wdir)

todo_fullpath = [os.path.join(wdir,txt) for txt in todo]

## Converting and saving the list of mean values into a XLS excel file.

In [6]:
data = tsgen.generate_time_series_datav2(wdir,todo)

series_df = pd.DataFrame(data=data)

series_df.to_excel(f'D:\\processing\\win\\{os.path.basename(out_dir)}.xlsx', sheet_name=f'{os.path.basename(out_dir)}')

Extracting image 1/171 - S3A_OL_2_WFR____20160514T134952...
Extracting image 2/171 - S3A_OL_2_WFR____20160518T134611...
Extracting image 3/171 - S3A_OL_2_WFR____20160606T135333...
Extracting image 4/171 - S3A_OL_2_WFR____20160610T134952...
Extracting image 5/171 - S3A_OL_2_WFR____20160618T134230...
Extracting image 6/171 - S3A_OL_2_WFR____20160626T133507...
Extracting image 7/171 - S3A_OL_2_WFR____20160630T133126...
Extracting image 8/171 - S3A_OL_2_WFR____20160704T132744...
Extracting image 9/171 - S3A_OL_2_WFR____20160707T134953...
Extracting image 10/171 - S3A_OL_2_WFR____20160711T134611...
Extracting image 11/171 - S3A_OL_2_WFR____20160715T134229...
Extracting image 12/171 - S3A_OL_2_WFR____20160719T133847...
Extracting image 13/171 - S3A_OL_2_WFR____20160727T133124...
Extracting image 14/171 - S3A_OL_2_WFR____20160730T135333...
Extracting image 15/171 - S3A_OL_2_WFR____20160803T134952...
Extracting image 16/171 - S3A_OL_2_WFR____20160807T134610...
Extracting image 17/171 - S3A_OL_

Extracting image 142/171 - S3B_OL_2_WFR____20190826T132526...
Extracting image 143/171 - S3A_OL_2_WFR____20190827T133845...
Extracting image 144/171 - S3B_OL_2_WFR____20190829T134752...
Extracting image 145/171 - S3A_OL_2_WFR____20190831T133501...
Extracting image 146/171 - S3B_OL_2_WFR____20190902T134408...
Extracting image 147/171 - S3B_OL_2_WFR____20190906T134024...
Extracting image 148/171 - S3B_OL_2_WFR____20190910T133639...
Extracting image 149/171 - S3A_OL_2_WFR____20190911T135000...
Extracting image 150/171 - S3A_OL_2_WFR____20190912T132349...
Extracting image 151/171 - S3B_OL_2_WFR____20190914T133255...
Extracting image 152/171 - S3A_OL_2_WFR____20190919T134232...
Extracting image 153/171 - S3A_OL_2_WFR____20190923T133848...
Extracting image 154/171 - S3A_OL_2_WFR____20190927T133504...
Extracting image 155/171 - S3B_OL_2_WFR____20190929T134407...
Extracting image 156/171 - S3A_OL_2_WFR____20191016T134234...
Extracting image 157/171 - S3B_OL_2_WFR____20191022T134749...
Extracti